# **GIỚI THIỆU**

* Xin chào! Hôm nay tôi sẽ phải hoàn thành một nhiệm vụ khá thú vị đó là **"Phân tích đánh giá cảm xúc người xem qua bộ dữ liệu 50K IMDB** bằng tất cả các **phương pháp học máy(gồm có: LSVC,Bayes Thơ Ngây,SGDC,LR,LSTM) và học sâu(CNN)** mà tôi biết. Vui lòng cho ý kiến khách quan để tất cả chúng ta có thể trao đổi với nhau tích cực hơn và hiệu quả hơn. Cảm ơn các bạn đã xem!

* Trước khi bắt đầu thì tôi sẽ cho bạn **biết trước kết quả thu được** sẽ ra sao để chúng ta có thể dễ dàng rà soát chúng một cách dễ dàng và hiệu quả.**(ở đây tôi chỉ trình bày kết quả đạt được về độ chính xác của từng mô hình, các bạn cảm thấy hay và hứng thú có thể xem từng chi tiết code & chú thích mà tôi để bên dưới).**

* Bạn có thể xem số liệu **"so sánh bảng mô hình" tại dòng "Nhận xét về kết quả thu được từ 4 mô hình" bên dưới**, tôi đã nén lại dưới dạng dataFrame cho bạn dễ nhìn & so sánh khách quan.

* ***Được rồi! chúng ta không vòng vo nữa, vào việc nào!Chúc bạn 1 ngày tốt lành...LOVE***

**KẾT QUẢ BÁO TRƯỚC(KẾT QUẢ IN ĐẬM LÀ KẾT QUẢ KIỂM TRA CÓ ĐỘ CHÍNH XÁC CAO NHẤT)**
1.  **LSVC:**
    * **Độ chính xác của việc sử dụng TF-IDF fit với mô hình Linear SVC: 90.29%**
    * Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Linear SVC: 89.77%
    * Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Linear SVC: 89.46%

2.  **MNB:**
    * Độ chính xác của việc sử dụng TF-IDF fit với mô hình Multinomial Naive Bayes: 86.63%
    * Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Multinomial Naive Bayes: 88.75%
    * Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit Multinomial Naive Bayes: 89.10%

3.  **SGDC:** 
    * TF-IDF: Độ chính xác của SGD Classifier: 88.67%
    * Độ chính xác của SGD Classifier binary = False: 85.84%
    * Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit SGD Classifier: 85.84%

4.  **LR:**
    * Độ chính xác của việc sử dụng TF-IDF fit với mô hình hồi quy logistic: 88.50%
    * Độ chính xác của việc sử dụng CountVectorizer fit với mô hình hồi quy logistic: 85.62%
    * Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit với mô hình hồi quy logistic: 89.65%
5.  **DEEP LEARNING**
    * **Mô hình CNN cho độ chính xác tốt nhất trong số tất cả với 84.71% trên tập dữ liệu train và test**
___

# **CÀI ĐẶT**

In [ ]:
!pip install nltk
!pip install spacy
!python -m spacy download en
!pip install scipy
!pip install -U scikit-learn
!pip install seaborn
!pip install tensorflow
!pip install tf-nightly

# **THƯ VIỆN CẦN THIẾT**

In [ ]:
import pandas as pd
import numpy as np
from numpy import array,asarray,zeros
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB

from keras.utils.np_utils import to_categorical 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import Embedding,LSTM
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.random.set_seed(4)

# **ĐỌC TẬP DỮ LIỆU VÀ HIỂN THỊ TẬP DỮ LIỆU**

In [ ]:
#df = pd.read_csv("../input/imdb-dataset/IMDB Dataset.csv",encoding="ISO-8859-1")
df = pd.read_csv("../input/imdb-dataset/IMDB Dataset.csv")
df.head()

In [ ]:
df.sentiment.value_counts()
#df.describe

***Ok! Như các bạn thấy sau khi đọc dữ liệu chúng ta có 50k cảm xúc, trong đó có 25k cảm xúc tích cực (Positive) và 25k cảm xúc tiêu cực(Negative). Nhưng đây chỉ là dữ liệu thô chúng ta có, việc chúng ta cần làm sẽ dự đoán lại các cảm xúc (sentiment) để có cái nhìn khách quan hơn và xem mô hình nào sẽ làm độ chính xác của dữ liệu được dự đoán là cao nhất.***

***Ok! Trước khi dự đoán mô hình chúng ta phải làm sạch dữ liệu (tiền xử lí dữ liệu). Để quá trình làm sạch dữ liệu diễn ra hiệu quả và quy mô với khối lượng dữ liệu lớn, chúng ta cần xây dựng hàm làm sạch dữ liệu ngẫu nhiên 1 câu trong tập dữ liệu. Được rồi! tới đây chắc bạn đã hiểu rồi. Tôi sẽ dùng dòng thứ 3 trong tập dữ liệu...loc[2]***

In [ ]:
example_sentences = df['review'].loc[2]
example_sentences

**TIỀN XỬ LÍ DỮ LIỆU CHO 1 CÂU**

In [ ]:
# 1. Xóa content HTML
soup = BeautifulSoup(example_sentences, "html.parser")
example_sentences = soup.get_text()
example_sentences

In [ ]:
# 2. Sử dụng Biểu thức chính quy để xóa mọi thứ
example_sentences = re.sub('\[[^]]*\]', ' ',example_sentences)
example_sentences = re.sub('[^a-zA-Z]', ' ',example_sentences)
example_sentences

In [ ]:
# 3. Chuyển qua từ thường(LowerCase)
example_sentences = example_sentences.lower()
example_sentences

In [ ]:
# 4. Tách từ (split)
example_sentences = example_sentences.split()
example_sentences

In [ ]:
# 5. Xử lí từ dừng(stopword)
example_sentences = [word for word in example_sentences if not word in set(stopwords.words('english'))]
example_sentences

In [ ]:
# 6. Stemming/Lemmatization
ps = PorterStemmer()
ps_example_sentences = [ps.stem(word) for word in example_sentences]
ps_example_sentences

In [ ]:
# 7. Lemmatization/WordNetLemmatizer
lem = WordNetLemmatizer()
example_sentences = [lem.lemmatize(word) for word in example_sentences]
example_sentences

In [ ]:
# 8. Nối lại các từ(join)
example_sentences = ' '.join(example_sentences)
example_sentences

***Hoàn hảo! Bạn thấy gì không? dòng thứ 3 trông Dataset đã được xử lí sạch sẽ...Ok! Mọi thứ có vẻ ổn! Chúng ta sẽ đến phần Vecto hóa văn bản***
___

# Vector hóa văn bản
   Tôi sẽ áp dụng 3 phương pháp cho công việc này như sau:

*   **CountVectorizer (Bag of Words Model-Mô Hình Túi Từ)**
*   **TfidfVectorizer (Bag of Words Model)**
*   **Keras Tokenizer (Embedding)--> Phần này tôi sẽ nói thêm Ở MỤC NHÚNG TỪ**

**CountVector & TF_IDF sẽ là 2 kĩ thuật mà tôi sẽ cài đặt,xử lí và kiểm tra cho các mô hình...**

In [ ]:
# Khởi tạo một kho(corpus) để chứa dữ liệu câu
corpus = []
corpus.append(example_sentences)

In [ ]:
# Vector hóa câu bằng mô hình túi từ (Bag of Words)--> chúng ta sẽ thấy có số 1,2 & 3...
dem_vecto = CountVectorizer()
sentences_dem_vecto = dem_vecto.fit_transform(corpus)
sentences_dem_vecto.toarray()

In [ ]:
# biến đổi túi từ thành 1- Vectorizer với Binary = True sẽ làm được việc đó
dem_vecto_bin = CountVectorizer(binary=True)
sentences_dem_vecto_bin = dem_vecto_bin.fit_transform(corpus)
sentences_dem_vecto_bin.toarray()

**Ok! Được rồi! Tiếp đến chúng ta sẽ tiếp cận sử dụng TfidfVectorizer (TF-IDF) & Tính (TF * IDF) của mô hình túi từ**

In [ ]:
tfidf_vecto = TfidfVectorizer()
sentences_tfidf_vecto = tfidf_vecto.fit_transform(corpus)
sentences_tfidf_vecto.toarray()

# Chia tập dữ liệu
**Tôi sẽ chia tập dữ liệu để huấn luận mô hình với 75% dữ liệu là traning, 25% là test, random_state = 42 (chúng ta có thể lấy random tùy ý) & sau đó, tôi cần thay thế "review" và "sentiment" về dạng 2 kiểu số là "Positive: 1" và "Nagative: 0 để fit với mô hình"**

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(df['review'], df['sentiment'], test_size=0.25, random_state=42)
y_train = (y_train.replace({'positive': 1, 'negative': 0})).values
y_test  = (y_test.replace({'positive': 1, 'negative': 0})).values
print(y_train)
print(y_test)

# **Tiền xử lí dữ liệu cho tập dữ liệu 50K**
**Ok! Mọi thứ gần ổn! Giờ đến lúc tôi sẽ cho làm sạch toàn tập dữ liệu... Tôi sẽ chia làm 2 phần.. Phần train và phần test để làm sạch... (^_^)vui lòng chờ vì dữ liệu lớn sẽ mất nhiều thời gian của bạn**

In [ ]:
corpus_train = []
for i in range(x_train.shape[0]):
    soup = BeautifulSoup(x_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)

In [ ]:
corpus_test  = []
for j in range(x_test.shape[0]):
    soup = BeautifulSoup(x_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)

In [ ]:
corpus_train[-1]

In [ ]:
corpus_test[-1]

In [ ]:
# Vectơ hóa bằng kỹ thuật TF-IDF - Phần này sử dụng chung cho mô hình
tfidf_vecto = TfidfVectorizer(ngram_range=(1, 3))
tfidf_vecto_train = tfidf_vecto.fit_transform(corpus_train)
tfidf_vecto_test = tfidf_vecto.transform(corpus_test)

# **TRIỂN KHAI MÔ HÌNH**
**Xong! Mọi thứ đã ổn...Giờ là lúc tôi xây dựng mô hình từng mô hình cho tập dữ liệu và đưa ra mô hình có độ chính xác cao nhất.**
___

   # Tôi sẽ bắt đầu với mô hình Linear Support Vector Classification(LSVC)

In [ ]:
LSVC = LinearSVC(C=0.5, random_state=42)
LSVC.fit(tfidf_vecto_train, y_train)
du_doan = LSVC.predict(tfidf_vecto_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn import metrics
print("Classification report of Linear SVC: \n",classification_report(y_test,du_doan, target_names = ['Positive','Negative']))
print("Confusion Matrix of Linear SVC: \n",confusion_matrix(y_test,du_doan))
dochinhxac = metrics.accuracy_score(y_test,du_doan)
print("Độ chính xác của việc sử dụng TF-IDF fit với mô hình Linear SVC: \n" +str('{:04.2f}'.format(dochinhxac *100)) + "%")

**Độ chính xác của việc sử dụng TF-IDF fit với mô hình Linear SVC: 
90.29%**

**Khoan! Tôi sẽ vectơ hóa bằng cách sử dụng CountVectorizer (binary = False) và fit với mô hình LinearSVC để kiểm tra xem thế nào?**

In [ ]:
dem_vecto = CountVectorizer(ngram_range=(1,3), binary=False)
false_dem_vecto_train = dem_vecto.fit_transform(corpus_train)
false_dem_vecto_test = dem_vecto.transform(corpus_test)

In [ ]:
false_dem_LSVC = LinearSVC(C=0.5, random_state=42, max_iter=5000)
false_dem_LSVC.fit(false_dem_vecto_train,y_train)
false_dem_du_doan_LSVC = false_dem_LSVC.predict(false_dem_vecto_test)

In [ ]:
print("Kết quả của việc sử dụng CountVectorizer (binary = False) và fit với mô hình LinearSVC\n")
print("Classification report of Linear SVC (binary = False): \n",classification_report(y_test,false_dem_du_doan_LSVC, target_names = ['Positive','Negative']))
print("Confusion Matrix of Linear SVC (binary = False): \n",confusion_matrix(y_test,false_dem_du_doan_LSVC))
accuracy_score_false_lsvc = metrics.accuracy_score(y_test,false_dem_du_doan_LSVC)
print("Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit với mô hình Linear SVC: \n" +str('{:04.2f}'.format(accuracy_score_false_lsvc *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit với mô hình Linear SVC: 89.77%**

***Bây giờ tôi sẽ vectơ hóa bằng cách sử dụng CountVectorizer (binary = True) và fit với mô hình LinearSVC để xem kết quả so với (binary = false) sẽ thế nào?***

In [ ]:
true_dem_vecto = CountVectorizer(ngram_range=(1,3), binary=True)
true_dem_vecto_train = true_dem_vecto.fit_transform(corpus_train)
true_dem_vecto_test = true_dem_vecto.transform(corpus_test)

In [ ]:
true_dem_LSVC = LinearSVC(C=0.5, random_state=42,max_iter=5000)
true_dem_LSVC.fit(true_dem_vecto_train,y_train)
true_dem_du_doan_LSVC = true_dem_LSVC.predict(true_dem_vecto_test)

In [ ]:
print("Kết quả của việc sử dụng CountVectorizer (binary = True) và fit với mô hình LinearSVC\n")
print("Classification report of Linear SVC binary = True: \n",classification_report(y_test,true_dem_du_doan_LSVC, target_names = ['Positive','Negative']))
print("Confusion Matrix of Linear SVC binary = True: \n",confusion_matrix(y_test,true_dem_du_doan_LSVC))
accuracy_score_true_LSVC = metrics.accuracy_score(y_test,true_dem_du_doan_LSVC)
print("Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit với mô hình Linear SVC: \n" +str('{:04.2f}'.format(accuracy_score_true_LSVC *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit với mô hình Linear SVC: 89.46%**

***TUYỆT VỜI! chúng ta đạt được độ chính xác gần như tối đa bằng cách sử dụng công cụ vectơ TF-IDF. Và cũng bằng công cụ này tôi sẽ sử dụng thêm mô hình thứ 2-Naive Bayes. Mô hình Bayes sử dụng các xác suất trước để dự đoán xác suất sau rất hữu ích cho việc phân loại với các tính năng rời rạc như phân loại văn bản.***
___
    
   # Mô hình thứ hai: Bayes Thơ Ngây(Multinomial Naive Bayes-MNB)

In [ ]:
tfidf_vecto_bayes = TfidfVectorizer(ngram_range=(1, 1))
tfidf_vecto_bayes_train = tfidf_vecto_bayes.fit_transform(corpus_train)
tfidf_vecto_bayes_test = tfidf_vecto_bayes.transform(corpus_test)
print(tfidf_vecto_bayes_train.toarray().shape,tfidf_vecto_bayes_test.toarray().shape)

**Có thể thấy rằng trong kho chúng ta có 81301 dữ liệu... Vì vậy, tôi đề xuất dùng Chi-Square(Chi Phân phối bình phương) khi dữ liệu của chúng ta ở dạng đếm tần suất và điều chúng ta quan tâm là số lượng đối tượng thuộc các loại khác nhau, Chi-Square sẽ giúp chúng ta thực hiện việc kiểm tra gần đúng về mặt ý nghĩa đối với sự liên kết(association) giữa hai biến lập nhóm (categorical variables). Bạn có thể lấy bao nhiêu features cũng được(ở đây tôi sẽ lấy ngẫu nhiên 50000 features)**

Các bạn có thể tham khảo Chi-Square tại: https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/using-chi-square-statistic-in-research/

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
chichi = SelectKBest(chi2, k=50000)
tfidf_vecto_bayes_train = chichi.fit_transform(tfidf_vecto_bayes_train,y_train)
tfidf_vecto_bayes_test = chichi.transform(tfidf_vecto_bayes_test)

In [ ]:
feat_names = tfidf_vecto_bayes.get_feature_names()
feat_names = [feat_names[i] for i in chichi.get_support(indices=True)]
feat_names = np.asarray(feat_names)
feat_names[32245]

**Ok! Giờ tôi sẽ fit nó với mô hình Multinomial Naive Bayes.**

In [ ]:
clf_MNB = MultinomialNB()
clf_MNB.fit(tfidf_vecto_bayes_train,y_train)
du_doan_MNB = clf_MNB.predict(tfidf_vecto_bayes_test)

In [ ]:
print("Kết quả của việc sử dụng TF-IDF fit với mô hình Multinomial Naive Bayes\n")
print("Classification report of Multinomial Naive Bayes: \n",classification_report(y_test,du_doan_MNB, target_names = ['Positive','Negative']))
print("Confusion Matrix of Multinomial Naive Bayes: \n",confusion_matrix(y_test,du_doan_MNB))
accuracy_score_MNB = metrics.accuracy_score(y_test,du_doan_MNB)
print("Độ chính xác của việc sử dụng TF-IDF fit với mô hình Multinomial Naive Bayes: \n" +str('{:04.2f}'.format(accuracy_score_MNB *100)) + "%")

**Độ chính xác của việc sử dụng TF-IDF fit với mô hình Multinomial Naive Bayes: 86.63%**

**Phân phối đa thức thường yêu cầu số feature số nguyên. Tuy nhiên, trong thực tế, các phép đếm phân số như tf-idf cũng có thể hoạt động. Bây giờ tôi sẽ sử dụng CountVectorizer làm mô hình Bag-of-Words trước khi áp dụng mô hình MultinomialNB để xem thế nào nha.**

***Tôi sẽ vectơ hóa bằng cách sử dụng CountVectorizer (binary = False) và fit với mô hình LinearSVC để kiểm tra xem thế nào?***

In [ ]:
vecto_MNB = CountVectorizer(ngram_range=(1, 3), binary=False)
dem_vecto_train_MNB = vecto_MNB.fit_transform(corpus_train)
dem_vecto_test_MNB = vecto_MNB.transform(corpus_test)

In [ ]:
dem_MNB = MultinomialNB()
dem_MNB.fit(dem_vecto_train_MNB,y_train)
dem_du_doan_MNB = dem_MNB.predict(dem_vecto_test_MNB)

In [ ]:
print("Kết quả của việc sử dụng CountVectorizer (binary = False) và fit với mô hình Multinomial Naive Bayes\n")
print("Classification report of Multinomial Naive Bayes binary = False: \n",classification_report(y_test,dem_du_doan_MNB, target_names = ['Positive','Negative']))
print("Confusion Matrix of Multinomial Naive Bayes binary = False: \n",confusion_matrix(y_test,dem_du_doan_MNB))
accuracy_score_dem_MNB = metrics.accuracy_score(y_test,dem_du_doan_MNB)
print("Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Multinomial Naive Bayes: \n" +str('{:04.2f}'.format(accuracy_score_dem_MNB *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Multinomial Naive Bayes: 88.75%**

In [ ]:
true_dem_vecto_MNB = CountVectorizer(ngram_range=(1,3), binary=True)
true_dem_vecto_train_MNB = true_dem_vecto_MNB.fit_transform(corpus_train)
true_dem_vecto_test_MNB = true_dem_vecto_MNB.transform(corpus_test)

In [ ]:
true_dem_MNB = MultinomialNB()
true_dem_MNB.fit(true_dem_vecto_train_MNB,y_train)
true_dem_du_doan_MNB = true_dem_MNB.predict(true_dem_vecto_test_MNB)

In [ ]:
print("Kết quả của việc sử dụng CountVectorizer (binary = True) và fit với mô hình Multinomial Naive Bayes\n")
print("Classification report of Multinomial Naive Bayes binary = True: \n",classification_report(y_test,true_dem_du_doan_MNB, target_names = ['Positive','Negative']))
print("Confusion Matrix of Multinomial Naive Bayes binary = True: \n",confusion_matrix(y_test,true_dem_du_doan_MNB))
accuracy_score_true_dem_MNB = metrics.accuracy_score(y_test,true_dem_du_doan_MNB)
print("Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit Multinomial Naive Bayes: \n" +str('{:04.2f}'.format(accuracy_score_true_dem_MNB *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit Multinomial Naive Bayes: 89.10%**
___

   # Mô hình thứ ba: Stochastic Gradient Descent Classifier(SGDC)
**Ok, CÓ thể bạn đang bị rối xíu... Ở mô hình thứ 3 này,để tiết kiệm thời gian cho bạn thì tôi sẽ gơm gọn code lại và đi nhanh phần TfidfVectorizervà CountVectorizer bằng cách sử dụng "binary = True" và "binary = False" luôn... Ok chứ! Bắt đầu nào!**

**CHÚ Ý: Từ mô hình số 3 này trở về sau, TF-IDF tôi sẽ sử dụng mặc định cài sẵn ở trên và code nhanh, tôi chỉ lặp lại phần countvectorier cho moi người dễ phân biệt...Ok!**

Bạn có thể đọc thên tại đây:https://scikit-learn.org/stable/modules/sgd.html

In [ ]:
#Count vectorizer for bag of words
dem_vecto_SGD = CountVectorizer(binary=False,ngram_range=(1,3))
dem_vecto_train_SGD=dem_vecto_SGD.fit_transform(corpus_train)
dem_vecto_test_SGD=dem_vecto_SGD.transform(corpus_test)
print('Count vectorizer for train:',dem_vecto_train_SGD.shape)
print('Count vectorizer for test:',dem_vecto_test_SGD.shape)

SGDC = SGDClassifier(loss='hinge',random_state=42)
false_dem_SGDC = SGDC.fit(dem_vecto_train_SGD,y_train)

tfidf_SGDC= SGDC.fit(tfidf_vecto_train,y_train)
#Predicting the model for bag of words
false_dem_du_doan_SGDC=false_dem_SGDC.predict(dem_vecto_test_SGD)
print(false_dem_du_doan_SGDC)
#Predicting the model for tfidf features
tfidf_du_doan_SGDC=tfidf_SGDC.predict(tfidf_vecto_test)
print(tfidf_du_doan_SGDC)

In [ ]:
print("Kết quả của việc sử dụng TF_IDF fit với mô hình SGD Classifier\n")
print("Classification report of SGD Classifier: \n",classification_report(y_test,tfidf_du_doan_SGDC, target_names = ['Positive','Negative']))
print("Confusion Matrix of SGD Classifier: \n",confusion_matrix(y_test,tfidf_du_doan_SGDC))
accuracy_score_tfidf_SGDC = metrics.accuracy_score(y_test,tfidf_du_doan_SGDC)
print("Độ chính xác của việc sử dụng TF_IDF fit với mô hình SGD Classifier: \n" +str('{:04.2f}'.format(accuracy_score_tfidf_SGDC *100)) + "%")

**Độ chính xác của việc sử dụng TF_IDF fit với mô hình SGD Classifier: 88.67%**

In [ ]:
print("Kết quả của việc sử dụng CountVectorizer (binary = False) fit với mô hình SGD Classifier\n")
print("Classification report of SGD Classifier: \n",classification_report(y_test,false_dem_du_doan_SGDC, target_names = ['Positive','Negative']))
print("Confusion Matrix of SGD Classifier: \n",confusion_matrix(y_test,false_dem_du_doan_SGDC))
accuracy_score_false_count_SGDC = metrics.accuracy_score(y_test,false_dem_du_doan_SGDC)
print("Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit với mô hình SGD Classifier: \n" +str('{:04.2f}'.format(accuracy_score_false_count_SGDC *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit với mô hình SGD Classifier: 85.84%**

**Bây giờ tôi sẽ vectơ hóa bằng cách sử dụng CountVectorizer (binary = True) và fit với mô hình SGDC để xem kết quả so với (binary = True) sẽ thế nào?**

In [ ]:
true_dem_vecto=CountVectorizer(binary=True,ngram_range=(1,3))
true_dem_vecto_train=true_dem_vecto.fit_transform(corpus_train)
true_dem_vecto_test=true_dem_vecto.transform(corpus_test)

true_SGDC = SGDClassifier(loss='hinge',random_state=42)
true_dem_SGDC=true_SGDC.fit(true_dem_vecto_train,y_train)

true_dem_du_doan_SGDC=true_dem_SGDC.predict(true_dem_vecto_test)
print("Kết quả của việc sử dụng CountVectorizer (binary = True) fit với mô hình SGD Classifier\n")
print("Classification report of SGD Classifier: \n",classification_report(y_test,true_dem_du_doan_SGDC, target_names = ['Positive','Negative']))
print("Confusion Matrix of SGD Classifier: \n",confusion_matrix(y_test,true_dem_du_doan_SGDC))
accuracy_score_true_count_SGDC = metrics.accuracy_score(y_test,true_dem_du_doan_SGDC)
print("Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit SGD Classifier: \n" +str('{:04.2f}'.format(accuracy_score_true_count_SGDC *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit SGD Classifier: 89.46%**
___

   # Mô hình thứ tư: Hồi Quy Logistic (Logistic regression)

   ***Cũng như mô hình thứ ba, tôi sẽ code nhanh phần này***

In [ ]:
LR_dem_vecto=CountVectorizer(binary=False,ngram_range=(1,3))
LR_false_dem_vecto_train=LR_dem_vecto.fit_transform(corpus_train)
LR_false_dem_vecto_test=LR_dem_vecto.transform(corpus_test)

LR=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
dem_LR=LR.fit(LR_false_dem_vecto_train,y_train)
tfidf_LR=LR.fit(tfidf_vecto_train,y_train)

false_du_doan_LR=dem_LR.predict(LR_false_dem_vecto_test)
du_doan_tfidf_LR=tfidf_LR.predict(tfidf_vecto_test)


In [ ]:
print("Kết quả của việc sử dụng TF-IDF fit với mô hình hồi quy logistic\n")
print("Classification report of Logistic Regression: \n",classification_report(y_test,du_doan_tfidf_LR, target_names = ['Positive','Negative']))
print("Confusion Matrix of Logistic Regression: \n",confusion_matrix(y_test,du_doan_tfidf_LR))
accuracy_score_tfidf_LR = metrics.accuracy_score(y_test,du_doan_tfidf_LR)
print("Độ chính xác của việc sử dụng TF-IDF fit với mô hình hồi quy logistic: \n" +str('{:04.2f}'.format(accuracy_score_tfidf_LR *100)) + "%")

**Độ chính xác của việc sử dụng TF-IDF fit với mô hình hồi quy logistic: 88.50%**

In [ ]:
print("Kết quả của việc sử dụng CountVectorizer (binary=False) fit với mô hình hồi quy logistic\n")
print("Classification report of Logistic Regression: \n",classification_report(y_test,false_du_doan_LR, target_names = ['Positive','Negative']))
print("Confusion Matrix of Logistic Regression: \n",confusion_matrix(y_test,false_du_doan_LR))
accuracy_score_false_count_LR = metrics.accuracy_score(y_test,false_du_doan_LR)
print("Độ chính xác của việc sử dụng CountVectorizer (binary=False) fit với mô hình hồi quy logistic: \n" +str('{:04.2f}'.format(accuracy_score_false_count_LR *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary=False) fit với mô hình hồi quy logistic: 85.62%**

In [ ]:
true_dem_LR_vecto=CountVectorizer(binary=True,ngram_range=(1,3))
true_dem_LR_vecto_train=true_dem_LR_vecto.fit_transform(corpus_train)
true_dem_LR_vecto_test=true_dem_LR_vecto.transform(corpus_test)

true_LR=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
true_dem_LR=true_LR.fit(true_dem_LR_vecto_train,y_train)

true_du_doan_LR=true_dem_LR.predict(true_dem_LR_vecto_test)

print("Kết quả của việc sử dụng CountVectorizer (binary = True) fit với mô hình hồi quy logistic\n")
print("Classification report of Logistic Regression: \n",classification_report(y_test,true_du_doan_LR, target_names = ['Positive','Negative']))
print("Confusion Matrix of Logistic Regression: \n",confusion_matrix(y_test,true_du_doan_LR))
accuracy_score_true_count_LR = metrics.accuracy_score(y_test,true_du_doan_LR)
print("Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit với mô hình hồi quy logistic: \n" +str('{:04.2f}'.format(accuracy_score_true_count_LR *100)) + "%")

**Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit với mô hình hồi quy logistic: 89.65%**
___

# **Nhận xét về kết quả thu được từ 4 mô hình:**

**Chúng ta có thể thấy rằng mô hình LinearSVC(LSVC) có TF-IDF đạt độ chính xác cao nhất:90.29%.** Vì vậy, tôi sẽ biểu diễn cho bạn thấy về kết quả dự đoán dự trên tập dữ liệu chúng ta có được. Ok! đi nào

In [ ]:
accuracy = {'LSVC': ['90.29','89.77','89.46'],
            'MNB': ['86.63','88.75','89.10'],
            'SGDC':['88.67','85.84','89.46'],
            'LR' :['88.50','85.62', '89.65']}
accuracy = pd.DataFrame(accuracy,index=['TF-IDF','Binary=False','Binary=True'])
accuracy

   # Biểu diễn kết quả dự đoán tập dữ liệu IMDB 50K

In [ ]:
df_predicted = x_test.copy()
df_predicted = pd.DataFrame(df_predicted)
df_predicted.columns = ['review']
df_predicted = df_predicted.reset_index()
df_predicted = df_predicted.drop(['index'], axis=1)
df_predicted.head()

In [ ]:
y_test_hientai = y_test.copy()
y_test_hientai = pd.DataFrame(y_test_hientai)
y_test_hientai.columns = ['sentiment']
y_test_hientai['sentiment'] = y_test_hientai['sentiment'].replace({1: 'positive', 0: 'negative'})

In [ ]:
y_test_predicted = du_doan.copy()
y_test_predicted = pd.DataFrame(y_test_predicted)
y_test_predicted.columns = ['predicted_sentiment']
y_test_predicted['predicted_sentiment'] = y_test_predicted['predicted_sentiment'].replace({1: 'positive', 0: 'negative'})

In [ ]:
ketqua_test = pd.concat([df_predicted, y_test_hientai, y_test_predicted], axis=1)
ketqua_test.head()

# Tạo lớp nhúng (Creation the embedding layer)
***Như đã trình bày, tôi cần mộp lớp nhúng chuyển đổi dữ liệu văn bản của tôi thành dữ liệu số. Nó được sử dụng làm lớp đầu tiên cho các mô hình học sâu trong keras. Đây là bước đầu tiên để tôi có thể tiến tới mô hình học sâu... Nó cũng khá nhanh và không mất nhiều thời gian của bạn đâu...cùng đi nhé!ok!Go!!!***

In [ ]:
# Tạo một từ để lập chỉ mục từ điển từ mô đun tokenizer 
# Mỗi từ được sử dụng làm khóa trong khi chỉ mục duy nhất tương ứng và được sử dụng làm giá trị cho khóa.
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)
 
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
#x_train
#print(y_train)
#print(y_test)
#x_test

In [ ]:
do_dai_danhsach_x = [len(i) for i in x_train + x_test]
print(f'Độ dài tối đa của câu:{max(do_dai_danhsach_x)}')
print(f'Độ dài trung bình của câu:{np.mean(do_dai_danhsach_x)}')

In [ ]:
# Sử dụng seaborn để vẽ độ dài câu
plt.figure(figsize=(8,5), dpi= 80)
sns.distplot(do_dai_danhsach_x, color='blue')
plt.title('Biểu đồ thể hiện số lượng từ',fontsize=20,color='black')
plt.xlabel('Số từ')
plt.ylabel('Số lượt review')
plt.show()

**Tôi sẽ đặt kích thước tối đa của mỗi danh sách là 100. Nếu danh sách nào có maxlength > 100 sẽ bị cắt bớt và danh sách nào có maxlength < 100 sẽ được thêm 0 cho đến khi có độ dài tối đa = maxlength.Dễ hiểu đúng không?**
**Quá trình này được gọi là padding. Đoạn code dưới đây sẽ tìm kích thước vocabulary và sau đó là perfom padding trên cả X_train và X_test , tokenizer.word_index**

In [ ]:
vocabulary_size = len(tokenizer.word_index) + 1
maxlength = 100

x_train_padding = pad_sequences(x_train, padding="post",maxlen=maxlength)
x_test_padding = pad_sequences(x_test, padding="post",maxlen=maxlength)
# Kiểm tra danh sách ngẫu nhiên trong X_train (các câu trước đó)
# có cùng độ dài là 100
len(x_train[3])

In [ ]:
# Kiểm tra xíu
x_train_padding

In [ ]:
vocabulary_size

* **Kết quả đã chỉ ra rằng tôi có 109137 từ duy nhất trong kho và tôi sẽ tạo mạng noron được thiết kế để từ dữ liệu số.**

* **Nhúng từ(Word Embedding) sẽ đảm nhiệm việc cải thiện khả năng mạng noron học từ dữ liệu văn bản và các vectơ được gọi chung là Embeddings.**

* **Tiếp theo tôi sẽ tạo ra ma trận feature mà cụ thể công cụ tôi sẽ sử dụng GloVe Embeddings. Theo định nghĩa, GloVe là viết tắt của Global Vectors dùng để biểu diễn từ. Đây là một thuật toán học không giám sát được phát triển bởi Stanford cho việc tạo nhúng từ bằng cách tổng hợp ma trận đồng xuất hiện các word-word toàn cục từ một kho ngữ liệu mà chúng ta thu được.**

* **Kết quả nhúng được hiển thị trong các cấu trúc tuyến tính thú vị của từ trong không gian vectơ. Nói cách khác,nếu hai từ cùng xuất hiện nhiều lần, điều đó có nghĩa là chúng có một số điểm tương đồng về ngôn ngữ hoặc ngữ nghĩa. Ok chứ!**
* **GloVe word Embedding sẽ tạo một từ điển và sẽ chứa các từ làm khóa(keys) và danh sách nhúng tương ứng của chúng dưới dạng giá trị(values)**

    * Bạn có thể tham khảo thêm tại đây: https://trituenhantao.io/kien-thuc/huong-dan-su-dung-glove/
    * Các bạn có thể tham khảo tại đây: https://ichi.pro/vi/nhung-tu-la-gi-71370589666818

In [ ]:
tudien_duoc_nhung = dict()
glove_file = open('../input/glove6b100d/glove.6B.100d.txt',encoding='utf-8')
for line in glove_file:
    record = line.split()
    word = record[0]
    kich_thuoc = asarray(record[1:], dtype = "float32")
    tudien_duoc_nhung[word] = kich_thuoc
glove_file.close()

**Kế tiếp, tôi sẽ tạo ra một ma trận nhúng(matrix embedding). Ở đây, mỗi số hàng sẽ tương ứng với chỉ mục của từ trong kho ngữ liệu. Ma trận sẽ chứa các embedding từ GloVe cho các từ(word) trong kho dữ liệu của tôi có được.Ok!**

In [ ]:
ma_tran_nhung = zeros((vocabulary_size,100))
for word, index in tokenizer.word_index.items():
    vecto_nhung = tudien_duoc_nhung.get(word)
    if vecto_nhung is not None:
        ma_tran_nhung[index] = vecto_nhung

In [ ]:
vecto_nhung

In [ ]:
ma_tran_nhung

In [ ]:
# Bạn có thể xóa dấu # để in ra 'tudien_duoc_nhung'
#tudien_duoc_nhung

# **Mô hình học Sâu (Deep Learning Models)**

***Ở mô hình học sâu thì nó hơi khác so với các mô hình máy học... Nhưng khi tìm hiểu thì mình thấy có rất nhiều thứ hay mà chúng ta có thể khám phá khi dự đoán cho tập dữ liệu chúng ta đang xét...Come on! cùng đi với tôi nào! GO!***

   # Mô hình học sâu đơn giản
* ***Để không bị rối thì tôi sẽ làm phần đơn giản cho bạn dễ hình dung mô hình học sâu được thiết kế thế nào? ok!***
* ***Tạo một mạng nơ-ron học sâu đơn giản. Sau đó tạo một lớp nhúng(class embed) bằng cách chỉ định các tham số mà chúng tôi đã tạo trước đó và thêm(add) nó vào mô hình(model)***
* ***Flatten lớp nhúng lại vì tôi đang kết nối thẳng lớp nhúng với lớp Dense(Dense layer)***
* ***Cuối cùng, tôi thêm một lớp dense với chức năng kích hoạt sigmoid.***
* ***Tôi biết bạn đang tự hỏi Sigmoid và Dense là gì? Chúng có quan hệ gì?***

*     Bạn có thể tham khảo tại đây: https://vn.got-it.ai/blog/tim-hieu-sigmoid-function-va-lich-su-hinh-thanh-cua-no
*     Bạn có thể tham khảo tại đây: https://trituenhantao.io/tu-dien-thuat-ngu/dense-layer/

In [ ]:
mo_hinh = Sequential()
lop_nhung = Embedding(vocabulary_size, 100, weights = [ma_tran_nhung],input_length=maxlength,trainable=False)
mo_hinh.add(lop_nhung)
mo_hinh.add(Flatten())
mo_hinh.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# Biên dịch
mo_hinh.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])
print(mo_hinh.summary())

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, batch_size=128,epochs = 20, verbose=1, validation_split=0.2)

In [ ]:
diem_danh_gia = mo_hinh.evaluate(x_test_padding, y_test, verbose = 1)

In [ ]:
#Kiểm tra độ chính xác
print("Độ chính xác: {:.2%}".format(diem_danh_gia[1]))

In [ ]:
du_doan_y = mo_hinh.predict(x_test_padding)
du_doan_y = np.round(du_doan_y).astype(int)
print(classification_report(y_test,du_doan_y))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Biểu đồ hiệu suất chính xác của mô hình',fontsize=15)
plt.xlabel('Epochs',fontsize=15)
plt.ylabel('Accuracy',fontsize=15)
plt.legend(['Train data','Test data'],loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Biểu đồ hiệu suất thất bại của mô hình',fontsize=15)
plt.xlabel('Epochs',fontsize=10)
plt.ylabel('Loss',fontsize=10)
plt.legend(['train','test'],loc='upper left')
plt.show()

***Độ chính xác trên tập dữ liệu huấn luyện cao hơn so với tập dữ liệu thử nghiệm nên mạng nơ-ron sâu không tốt cho các dự đoán.**

   # Mạng nơ-ron hồi quy (RNN - Recurrent Neural Network) + LSTM
   
   **Mạng nơ-ron hồi quy hoạt động tốt trên dữ liệu sequence. Dữ liệu văn bản là một chuỗi sequence data nên RNN là lựa chọn tốt để giải quyết các vấn đề liên quan đến văn bản.**
* **Thứ nhất, tôi xây dựng cấu trúc mô hình và tạo một lớp nhúng bằng cách chỉ định các tham số mà chúng ta đã tạo trước đó.**
* **Tiếp theo,tạo mạng nơ-ron lặp lại. Ở đây, tôi sẽ sử dụng LTSM-Long Short Term Memory networks(Mạng bộ nhớ dài-ngắn).**
* **Hai chiều có nghĩa là trình tự xử lý RNN từ đầu đến cuối và ngược lại Điều này sẽ làm cho mô hình hoạt động tốt hơn.ôi đã thêm một lớp ẩn khác và bao gồm một chức năng kích hoạt là relu.s**
* **Cuối cùng, tôi thêm một lớp dense với chức năng kích hoạt sigmoid.**

* Bạn có thể tham khảo tại đây:https://nttuan8.com/bai-14-long-short-term-memory-lstm/
* Bạn có thể tham khảo tại đây: https://dominhhai.github.io/vi/2017/10/what-is-rnn/

In [ ]:
mo_hinh = tf.keras.Sequential([tf.keras.layers.Embedding(vocabulary_size, 100,weights=[ma_tran_nhung],input_length=maxlength, trainable=False),
                                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),tf.keras.layers.Dense(128, activation='relu'),
                                tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
mo_hinh.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(1e-4),metrics=['accuracy'])

In [ ]:
mo_hinh.summary()

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs=10,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
acc_score = mo_hinh.evaluate(x_test_padding, y_test, verbose = 1)
acc_score

In [ ]:
print("Accuracy Test: {:.2%}".format(acc_score[1]))
print("Accuracy Score: {:.2%}".format(acc_score[0]))

***Độ chính xác của tập dữ liệu huấn luyện và kiểm tra gần nhau, vì vậy mô hình RNN (LSTM) hoạt động tốt trên dữ liệu mới.**

In [ ]:
# hàm dự đóán
du_doan_y = mo_hinh.predict(x_test_padding)
du_doan_y = np.round(du_doan_y).astype(int)
print(classification_report(y_test,du_doan_y))

In [ ]:
plt.plot(history.history['accuracy'],color="red")
plt.plot(history.history['val_accuracy'])
plt.title("Hiệu Suất Chính Xác Mô Hình Sử Dụng RNN",fontsize=15)
plt.xlabel("Epochs",fontsize=10)
plt.ylabel("Accuracy",fontsize=10)
plt.legend(["Train Data","Text data"],loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'],color="red")
plt.plot(history.history['val_loss'])
plt.title("Hiệu Suất Thất Bại Mô Hình Sử Dụng RNN",fontsize=15)
plt.xlabel("Epochs",fontsize=10)
plt.ylabel("loss",fontsize=10)
plt.legend(["train Data","text data"],loc='upper left')
plt.show()

   # Mạng Nơ-ron Hồi Quy Với (2 RNN layer or 2 LSTM layer) + dropout
   **TƯƠNG TỰ như trên: Nhưng ở đây tôi thêm vào DROPOUT, ta được:**
* **Thứ nhất, tôi xây dựng cấu trúc mô hình và tạo một lớp nhúng bằng cách chỉ định các tham số mà chúng ta đã tạo trước đó.**
* **Tiếp theo,tạo mạng nơ-ron lặp lại. Ở đây, tôi sẽ sử dụng LTSM-Long Short Term Memory networks(Mạng bộ nhớ dài-ngắn).**
* **Hai chiều có nghĩa là trình tự xử lý RNN từ đầu đến cuối và ngược lại Điều này sẽ làm cho mô hình hoạt động tốt hơn.ôi đã thêm một lớp ẩn khác và bao gồm một chức năng kích hoạt là relu**
* **thêm Dropout để ngăn mô hình không vừa vặn**
* **loại bỏ ngẫu nhiên một số tế bào thần kinh trong các lớp ẩn**
* **Cuối cùng, tôi thêm một lớp dense với chức năng kích hoạt sigmoid.**

In [ ]:
mo_hinh = tf.keras.Sequential([tf.keras.layers.Embedding(vocabulary_size, 100,weights=[ma_tran_nhung],input_length=maxlength, trainable=False),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences = True)),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(25)),
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
mo_hinh.summary()

In [ ]:
mo_hinh.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(1e-4),metrics=['accuracy'])

**Lắp mô hình với bộ train để training và tôi sử dụng 30 Epochs để tìm độ chính xác tối ưu nhất có thể.Ok! Đợi tí xem kết quả nào!**

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs= 10,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs= 5,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs= 5,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs= 2,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs= 2,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs= 1,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, epochs= 1,batch_size = 128, verbose = 1,validation_split = 0.2)

In [ ]:
acc_score = mo_hinh.evaluate(x_test_padding, y_test, verbose = 1)
acc_score

In [ ]:
print("Accuracy Score: {:.2%}".format(acc_score[0]))
print("Accuracy Train:{:.2%}".format(acc_score[1]))

In [ ]:
du_doan_y = mo_hinh.predict(x_test_padding)
du_doan_y = np.round(du_doan_y).astype(int)
print(classification_report(y_test,du_doan_y))

**Độ chính xác khi sử dụng lớp RNN bổ sung thêm LSTM(2 lớp RNN + LSTM) + propout  là 83%.**

   # Convolutional Neural Network with dropout

In [ ]:
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D,Conv1D,LSTM

mo_hinh = Sequential()

lop_nhung = Embedding(vocabulary_size, 100, weights=[ma_tran_nhung], input_length=maxlength , trainable=False)
mo_hinh.add(lop_nhung)

mo_hinh.add(Conv1D(128, 5, activation='relu'))
mo_hinh.add(GlobalMaxPooling1D())
mo_hinh.add(Dropout(0.2)),
mo_hinh.add(Dense(1, activation='sigmoid'))

In [ ]:
mo_hinh.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
print(mo_hinh.summary())

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)
accuracy_score = mo_hinh.evaluate(x_test_padding, y_test, verbose=1)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, batch_size=128, epochs=4, verbose=1, validation_split=0.2)

accuracy_score = mo_hinh.evaluate(x_test_padding, y_test, verbose=1)

In [ ]:
history = mo_hinh.fit(x_train_padding, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

accuracy_score = mo_hinh.evaluate(x_test_padding, y_test, verbose=1)

In [ ]:
du_doany = mo_hinh.predict(x_test_padding)
du_doany = np.round(du_doany).astype(int)
print(classification_report(y_test,du_doany))

In [ ]:
accuracy_score

In [ ]:
print("Test Score: {:.2%}".format(accuracy_score[0]))
print("Test Accuracy: {:.2%}".format(accuracy_score[1]))

**Độ chính xác khi sử dụng CNN trên tập dữ liệu là 84.71%**

# Word cloud for positive review words

In [ ]:
from wordcloud import WordCloud 
plt.figure(figsize=(15,5))
positive_text = corpus_train[2]
WC=WordCloud(width=1500,height=1000,max_words=500,min_font_size=5)
positive_words=WC.generate(positive_text)
plt.imshow(positive_words,interpolation='bilinear')
plt.show

In [ ]:
from wordcloud import WordCloud 
plt.figure(figsize=(15,5))
positive_text = corpus_train[20]
WC=WordCloud(width=1500,height=1000,max_words=500,min_font_size=5)
positive_words=WC.generate(positive_text)
plt.imshow(positive_words,interpolation='bilinear')
plt.show

   # Tạo trình tạo đánh giá ngẫu nhiên

* **Tạo một biến cho bài đánh giá phim được tạo ngẫu nhiên**
* **in bài đánh giá ngẫu nhiên**

In [ ]:
index = np.random.randint(x_train_padding.shape[0])
ex = corpus_train[index]
print(ex)

***Ở đây,Tôi muốn dự đoán đánh giá ngẫu nhiên là poitive or negative(tích cực hoặc tiêu cực) bằng cách sử dụng mô hình đã tạo. Vì vậy, bài đánh giá sẽ được xử lý trước tương tự như những gì mà tôi đã làm trong tập dữ liệu trước khi huấn luyện, tức là các từ sẽ được chuyển đổi thành số nguyên và độ dài tối đa là 100. Danh sách trống được tạo ra và sẽ lưu trữ các giá trị. Cuối cùng, tôi tạo một vòng lặp for để lặp qua danh sách. Chúng ta sẽ dự đoán được đánh giá người xem thế nào?***

**QUY ƯỚC:**

* **Dự đoán ngẫu nhiên gần bằng 1 => Đánh giá tích cực(Positive)**
* **Dự đoán ngẫu nhiên = 0 => Đánh giá tiêu cực(Negative)**

In [ ]:
ex = tokenizer.texts_to_sequences(ex)
ds = []
for e in ex:
    for i in e:
        ds.append(i)
ds = [ds]
ex = pad_sequences(ds, padding = 'post', maxlen = maxlength)
mo_hinh.predict(ex)

**=> Đánh giá ngẫu nhiên tôi tạo ra cho kết quả 0,99 gần bằng 1, có nghĩa là đó là đánh giá TÍCH CỰC(POSITIVE).**

# Kết Luận
* **Cả 5 giải thuật thì giải thuật hiểu quả nhất là Linear SVC với độ chính xác 90.29 %**
* **Tất cả các giải thuật còn lại điều hoạt động tốt và hiệu quả cũng khá cao**
* **Việc tối ưu hóa mô hình có thể mang lại kết quả tốt hơn khi sử dụng nhiều dữ liệu hơn.**
* **Độ chính xác từng mô hình**
    * **1.  LSVC:**
        * **Độ chính xác của việc sử dụng TF-IDF fit với mô hình Linear SVC: 90.29%**
        * Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Linear SVC: 89.77%
        * Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Linear SVC: 89.46%

    * **2.  MNB:**
        * Độ chính xác của việc sử dụng TF-IDF fit với mô hình Multinomial Naive Bayes: 86.63%
        * Độ chính xác của việc sử dụng CountVectorizer (binary = False) fit Multinomial Naive Bayes: 88.75%
        * Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit Multinomial Naive Bayes: 89.10%

    * **3.  SGDC:**
        * TF-IDF: Độ chính xác của SGD Classifier: 88.67%
        * Độ chính xác của SGD Classifier binary = False: 85.84%
        * Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit SGD Classifier: 85.84%

    * **4.  LR:**
        * Độ chính xác của việc sử dụng TF-IDF fit với mô hình hồi quy logistic: 88.50%
        * Độ chính xác của việc sử dụng CountVectorizer fit với mô hình hồi quy logistic: 85.62%
        * Độ chính xác của việc sử dụng CountVectorizer (binary = True) fit với mô hình hồi quy logistic: 89.65%
        
* **Ngoài ra,việc sử dụng phương pháp học sâu thì:**
        * Độ chính xác khi sử dụng CNN trên tập dữ liệu là 84.71%
        * Điểm chính xác khi sử dụng mạng nơ-ron học sâu đơn giản là 71.80%%
        * Độ chính xác khi sử dụng lớp RNN bổ sung thêm LSTM(2 lớp RNN + LSTM) là 83%.
        * Độ chính xác khi sử dụng 2 lớp RNN với LSTM là 80.89%
        * Mô hình CNN cho độ chính xác tốt nhất trong số tất cả với 84.71% trên tập dữ liệu train và test.
        * Sử dụng RNN (LSTM) với hai lớp ẩn cũng là cách tốt nhất mang lại độ chính xác 80.89% trên cả tập dữ liệu đào tạo và kiểm tra.
     * **Mô hình Mạng Nơ-ron hiệu quả để phân tích cảm xúc trên các đánh giá IMDB**

# GOOD BYE (^_^) Thanks you very much, Thanks come from BAO BI!

**Class A1, Computer Science, Can Tho University, Vietnam**

**FB:** https://www.facebook.com/baobi1998

**Kaggle:** https://www.kaggle.com/lienchibaob1812254

**github:** https://github.com/lienchibao1998